# Attention 前世今生

# 目录

- [参考文献](#参考文献)
- [Attention作用](#Attention作用)
- [Attention的提出](#Attention的提出)
    - [背景](#背景)
    - [Attention_in_Seq2Seq](#Attention_in_Seq2Seq)
- [Attention提出的意义](#Attention提出的意义)
- [有趣的联想](#有趣的联想)
- [通用定义](#通用定义)
- [计算步骤](#计算步骤)
- [Attention的多种形式](#Attention的多种形式)
    - [Local_Attention](#Local_Attention)
    - [Self-Attention](#Self-Attention)
    - [Transformer——集Attention大成者](Transformer——集Attention大成者)


# 参考文献

- https://zhuanlan.zhihu.com/p/91315967


# 作用

Attention机制可以让神经网络**更多的关注**到输入中**相关的信息**，并**减少对无关信息的注意**。帮助神经网络**更好的利用输入的信息**，有助于提升模型的可解释性（例如，词语间的相关性可视化）。

## Attention的提出

## 背景

![](../imgs/seq2seq1.jpg)

上图seq2seq模型存在问题：尤其输入长句时，Encoder输出Context向量给Decoder，由于Context长度有限，所以难以充分捕捉输入句子的信息。



## Attention_in_Seq2Seq

![Attention机制在Seq2Seq中应用的示意图](../imgs/attseq2seq.jpg)



上面左图：

Decoder的输入$s_1$（seq2seq中的context）和Encoder中的$h_1,h_2,h_3$进行计算，得到相应的attention scores，然后得到attention distribution。从图中可以看出，$s_1$对“好”字更敏感，分布值也最高，输出的attention output(即context向量)$c_1$中$h_3$的比例最高，对应的由$c_1$拼接$s_1$而成的新向量在Decoder中映射出的单词是“good”。

对比上一张图中seq2seq，attention seq2seq使用$c_1$拼接$s_1$作为context向量，增加了向量的信息量。

上面右图：

左图中（即上一step）decoder输出的结果“good”又作为decoder的输入，得到输入向量$s_2$，输入$s_2$和Encoder中的$h_1,h_2,h_3$进行计算，分别得到attention scores$e_1,e_2,e_3$，在经过$softmax$计算得到attention distribution，分布值分别为$α_1,α_2,α_3$。$α_1,α_2,α_3$再分别与$h_1,h_2,h_3$相乘并sum结果，得到attention outpu$c_2$.可以看到“早上”二字的分布值较大，在$s_2$对应的context向量中占较大比例，对应step输出为"morning"。

继续如此到“enb”为止。

attention output（context向量）计算公式如下：

$$c_i = \sum_{j=1}^{T_x}α_{ij}h_j$$

其中$i$表示decoder的第i个输入，$j$表示在encoder输出的第$j$个隐藏变量。$c_i$是decoder的第$i$个输入对应的context向量（decoder第i个输入与encoder每个时间步隐层向量的加权和，对应的权重为$α_{ij}$），$h_j$是encoder输出的第$j$个隐藏层变量，$α_{ij}$是decoder第$i$个输入与encoder第$j$个隐藏层变量对应的权重。

注意力权重(注意力分布值)计算公式：

$$α_{ij}=\frac{exp(e_{ij})}{\sum_{k=1}^{T_x}exp(e_{ik})}$$

注意力分数$e_{ij}$表示在decoder运行到第$i$时部时，对应于encoder第$j$个序列隐藏层输出的注意力分数计算公式如下：

$$e_{ij}=v_a^Ttanh(W_as_i + U_ah_j)$$

其中$s_i$为decoder第$i$时部的隐层向量，$h_j$是encoder的第$j$个隐层向量。$v_a$是一个向量，用于将向量转化为实数。

实例如下：

![ed例子](../imgs/rnned1.jpg)

# Attention提出的意义

- Attention机制解决了定长context向量导致的信息瓶颈问题
- 缓解RNN（包括LSTM）在长距离依赖中梯度消失的问题（Decoder可以自由的对RNN的各个时步直接建立连接）
- 提供了一定的可解释性（论文中给出的对齐矩阵如下图）。

![对齐矩阵](../imgs/attmat.jpg)

# 有趣的联想

在LSTM和GRU， 以及ResNet中，其实已经蕴藏着attention的思想。用**加法(sum)**的形式将**参数(weight)**以**不同的权重(score)**组合起来，就是attention的精髓了。

只不过这些模型中权重是由各种门决定(Gate)，注意到门的输出也是**(0, 1)区间**，和Attention权重由Softmax输出在(0, 1)之间也保持一致。

且他们（Gate与Attention层）均为joint learning，即通过和目标一起联合学习得到。

# 通用定义

**给出一组值向量(values)和一个查询向量(query)，attention是一种根据查询向量(query)计算这组值向量(values)的加权和的方法。**

例如在Seq2Seq中，encoder各个时步的隐层向量$h_j$组成了我们的值向量(values)，而decoder的在第i步输入时的隐层向量 $s_i$ 是我们的查询向量(query)。我们即是根据这个 $s_i$ 计算出我们全体 $h_j$ 的加权和，这个加权和即为我们的Attention输出。

更进一步，这个**加权和就是对values中的信息的一个有选择性的概要，我们的query决定了values中的哪一部分会被关注，会被着重提取**。

同时Attention也是一种获取一组长度不定的向量(values)依赖于另一个向量(query)的定长表示的方式。

# 计算步骤

假设有一组向量(values)$h_1,h_2,h_3,..,h_N \in R^{d_h}$以及一个查询向量(query)$s \in R^{d_s}$,计算attention需要以下三个步骤：

1. 计算attention scores $e \in R^N$  (这一步的实现方法很多)
2. 通过softmax获得权重 $α=softmax(e) \in R^N$
3. 根据得到对权重分布计算加权和(context向量) $c=\sum_{i=1}^{N}α_ih_i \in R^{d_h}$


【相关计算注意力得分公式未列出，请在原文中找 https://zhuanlan.zhihu.com/p/91315967 】

# Attention的多种形式

## Self-Attention

在文本分类，文本推荐等领域（N to 1），虽然输入是一个序列（或者说一组向量），但是输出却不是（输出通常为一个值）。我们有values，但是似乎难以寻找一个额外的query。例如文本分类中，除了文本本身并没有其他输入。

针对这样的场景， Yang et al.在2016年提出了Self Attention [ Yang et al. 2016]。 顾名思义，**self attention的query和values都属于同一个序列**。

观察到**对于一个句子的向量表示，各个词在其中的贡献程度都是不一样的。同样对于一篇文章的向量表示而言，各个句子对其的贡献程度也是不同的。**Yang et al.希望**通过Self attention机制可以帮助提取出相对更重要的词语或句子**。

![selfattention](../imgs/selfatt.jpg)

这里 $u_s$可以理解为对于“哪一句话携带最多能帮助文章分类信息”这个问题(或者说query)的一种向量表示，可以一开始对 $u_s$进行随机初始化，然后再进行joint learning, 这里可以参考Sukhbaatar et al.对meomory network中问题文本的向量化处理 [Sukhbaatar, et al. 2015].

## Transformer——集Attention大成者